In [1]:
import pandas as pd
import itertools
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [2]:
before_covid_umap_df = pd.read_excel('before_covid_umap_df(20) (2).xlsx')

In [3]:
llm = ChatOpenAI(
    temperature=0,
    openai_api_key=""
)

In [4]:
map_template = """The following is a set of documents
{docs}
Based on this list of docs, you should identify the main themes in Korean.
Instead of simply listing the content, please provide the overarching themes based on the technologies and fields that run through each docs in three sentences or less.

This is a sample answer : 주어진 문서는 초지 이용 기술 개발과 체험관광형 목장 이용 모델 설정을 통해 초지의 부가가치 창출과 방문객 만족도 향상을 도모하고자 하는 연구 내용을 담고 있습니다.

Helpful Answer:"""

prompt_template = PromptTemplate.from_template(
    template=map_template
)

map_chain = LLMChain(llm=llm, prompt=prompt_template)

In [5]:
reduce_template = """The following is set of summaries in Korean:
{docs}
Take these and distill it into a final, consolidated summary of the main themes in Korean.
Instead of simply listing the content, please provide the overarching themes based on the technologies and fields that run through each docs in three sentences or less.

This is a sample answer : 주어진 문서는 초지 이용 기술 개발과 체험관광형 목장 이용 모델 설정을 통해 초지의 부가가치 창출과 방문객 만족도 향상을 도모하고자 하는 연구 내용을 담고 있습니다.

Helpful Answer:"""

reduce_prompt = PromptTemplate.from_template(reduce_template)

In [6]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain
    # The maximum number of tokens to group documents into.
#    token_max=4000,
)

In [7]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True)

In [9]:
cluster_1 = list(before_covid_umap_df.loc[before_covid_umap_df['cluster']==1, '초록(국문)'])

In [10]:
docs = text_splitter.create_documents(cluster_1)

In [11]:
map_reduce_chain.run(docs)

'문서들의 주요 테마는 핵융합 분야에서 대학과 대학원 수준의 온라인 비디오 강의 개발과 홍보입니다. 이를 통해 학생들에게 핵융합 분야의 플라즈마 기초에 대해 깊이 있는 학습 기회를 제공하고 이 분야로의 진입을 용이하게 할 것입니다. 또한, 문서들은 ECSS-E-40과 DO-178C의 요구 사항을 충족하기 위해 아키텍처 기반의 순환 공학 시각화 모델 및 플랫폼 구현, 형식적 방법을 사용한 순환 공학 환경 개발 등을 포함한 우주 소프트웨어의 구성 관리, 품질 보증, 테스트 및 평가, 실패 및 이력 관리, 안전 및 신뢰성 보증에 대한 방법론과 도구의 개발에 대해 논의합니다. 또한, 문서들은 노인들의 추락 위험 평가를 위한 스크리닝 방법론의 개발과 검증, 장기요양보험과 관련된 의료비 절감을 위한 시스템화된 평가 방법의 개발 및 기존 추락 위험 평가 방법과의 통합을 목표로 합니다. 이러한 문서들은 노인들의 추락 예방 전략과 노인의 장기요양에 따른 의료비 절감에 관한 주제를 다룹니다.'